In [10]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv

load_dotenv()

llm=HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation"
)

model=ChatHuggingFace(llm=llm)

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

model=ChatGoogleGenerativeAI(model='gemini-1.5-flash-latest', temperature=0)


In [18]:
from langchain_core.prompts import PromptTemplate

template1= PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variable={'topic'}
)

template2= PromptTemplate(
    template="Write a 5 line summary on the following text{topic} and number them",
    input_variable={'topic'}
)

prompt1=template1.invoke({"topic":"Black Hole"})
result = model.invoke(prompt1)

prompt2=template2.invoke({"topic":result.content})
result2= model.invoke(prompt2)

print(result2.content)




1. Black holes, formed from collapsed massive stars, possess immense gravity warping spacetime, crucial for testing General Relativity and understanding galactic evolution.
2.  Several types exist: stellar-mass, supermassive, intermediate-mass (less understood), and hypothetical primordial black holes.
3. Key properties include mass, spin, (generally negligible) electric charge, the event horizon (escape boundary), and the singularity (point of infinite density).
4. Detection relies on indirect observation: gravitational lensing, accretion disk radiation, and gravitational waves from mergers.
5. Ongoing research focuses on formation mechanisms, singularity physics, galactic influence, primordial black hole searches, and improved detection methods.


str output Parsers

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template1= PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variable={'topic'}
)

template2= PromptTemplate(
    template="Write a 5 line summary on the following text {topic} and number them ",
    input_variable={'topic'}
)

parser= StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result=chain.invoke({"topic":"Black Hole"})

print(result)

1. Black holes are regions of spacetime with immense gravity, preventing even light from escaping beyond their event horizon.
2. They form from the gravitational collapse of massive stars, existing in stellar-mass, supermassive, intermediate-mass, and potentially primordial forms.
3. Key properties include mass, spin, and the singularity at their center; detection relies on observing their gravitational effects and radiation emissions.
4. Current research focuses on supermassive black hole formation, singularity nature, galactic influence, primordial black hole searches, and merger studies.
5. Black hole research is crucial for understanding gravity, spacetime, and the universe's evolution, with ongoing discoveries continually expanding our knowledge.


JSON Output Parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

template = PromptTemplate(
    template="Give me the name, age and city of a fictional person\n{format_instruction}",
    input_variables=[],
    partial_variables={"format_instruction": json_parser.get_format_instructions()}
)

prompt = template.format()
print(prompt)

result=model.invoke(prompt)

output=json_parser.parse(result.content)

""" Instead of doing the 3 steps we can also use chain
chain= template|model|parser
result=chain.invoke({})
"""

print(output)

Give me the name, age and city of a fictional person
Return a JSON object.
{'name': 'Evelyn Reed', 'age': 32, 'city': 'Portland, Oregon'}


Structured Output Parsers

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schema= [
    ResponseSchema(name="fact1", description="Fact 1 about the topic"),
    ResponseSchema(name="fact2", description="Fact 2 about the topic"),
    ResponseSchema(name="fact3", description="Fact 3 about the topic")

]

parser= StructuredOutputParser.from_response_schemas(schema)

template= PromptTemplate(
    template="give 3 facts about {topic}\n{format_instruction}",
    input_variables=["topic"],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

# prompt= template.invoke({'topic':'BlackHole'})

# result= model.invoke(prompt)

# final_result=parser.parse(result.content)

# print(final_result)

chain= template| model| parser 
result = chain.invoke({"topic":"BlackHole"})

print(result)


{'fact1': 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape.', 'fact2': 'Black holes are formed from the remnants of massive stars that have collapsed under their own gravity.', 'fact3': 'The singularity at the center of a black hole is a point of infinite density and zero volume, where the known laws of physics break down.'}


Pydantic Output Parsers

In [36]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str= Field(description="Name of the person")
    age: int= Field(gt=18, description="Age of the person")
    city: str= Field(description="City name where person lives")

parser= PydanticOutputParser(pydantic_object=Person)

template= PromptTemplate(
    template= "Generate a name, age and city of a fictional character from {place}\n {format_instruction}",

    input_variables=["place"],
    partial_variables={'format_instruction': parser.get_format_instructions()}

)

chain = template| model| parser

result= chain.invoke({"place":"africa"})

print(result)
print(type(result))


name='Aisha Mohammed' age=28 city='Nairobi'
<class '__main__.Person'>
